In [14]:
import requests
import urllib
from datetime import datetime
import pandas as pd
import time
import asyncio

BASE_URL = 'https://api.prop-odds.com'
API_KEY = 'qL5VXnhAyK6cLbKzpD45yfLKdMzoCcepZXQJWs2o'

#### first four methods are adapted from prop-odds @ https://github.com/M4THYOU/prop_odds_python_example/blob/main/main.py
def get_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

    print('Request failed with status:', response.status_code)
    return {}


def get_nba_games():
    now = datetime.now()
    query_params = {
        'date': now.strftime('%Y-%m-%d'),
        'tz': 'America/New_York',
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/games/nba?' + params
    return get_request(url)


def get_most_recent_odds(game_id, market):
    query_params = {
        'api_key': API_KEY,
        'active_only': False,
        'end_date_time': "2024-01-08T00:00:00Z"
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/odds/' + game_id + '/' + market + '?' + params
    return get_request(url)


def get_fantasy_lines(game_id, market):
    query_params = {
        'api_key': API_KEY,
        'active_only': False,
        'end_date_time': "2024-01-08T00:00:00Z"
    }

    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/fantasy_lines/' + game_id + '/' + market + '?' + params
    return get_request(url)


In [5]:
#### processing each book

## for 0:

def process_zero(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]
    df[['participant_name', 'name']] = df['name'].str.rsplit(' ', 1, expand=True)

    df = df.dropna()
    return df

## for 1:

def process_one(df):
    df = df.copy()
    df.dropna()
    return df[['handicap', 'odds', 'participant_name', 'name']]

## for two and six

def process_two_six(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['name', 'participant_name']] = df['name'].str.split(' - ', expand=True)
    df['participant_name'] = df['participant_name'].str.strip()

    df.dropna()

    return df

## for three and five

def process_three_five(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name', 'description']]

    df[['participant_name', 'description', 'a']] = df['description'].str.rsplit(' ', n=2, expand=True)
    df['participant_name'] = df['participant_name'].str.strip()

    df = df.drop(columns=['description', 'a'])
    df = df.dropna()

    return df


## for seven and eight

def process_seven_eight(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['participant_name', 'name', 'a']] = df['name'].str.rsplit(' ', n=2, expand=True)

    df = df.drop(columns=['a'])
    df = df.dropna()

    return df

In [6]:
#### for merging all the books

## agg
def agg(df):    
    result_df = df.groupby(['participant_name', 'name']).agg({'handicap': 'first', 'odds': 'median'}).reset_index()
    return result_df

## pivot
def pivot(df):
    df = df.copy()
    pivot_df = pd.pivot_table(df, values='odds', index=['handicap', 'participant_name'], columns='name').reset_index()
    pivot_df.columns = ['line', 'participant_name', 'over_odds', 'under_odds']

    return pivot_df

## calculate odds
## adapted from Ammar Sulmanjee

def calculate_odds(x, y):
    if x >= 0:
        decimal_odds_1 = 1 + x/100
    else: 
        decimal_odds_1 = 1 + 100/abs(x)
        
    if y >= 0:
        decimal_odds_2 = 1 + y/100
    else: 
        decimal_odds_2 = 1 + 100/abs(y)

    imp_prob1 = (1 / decimal_odds_1) * 100
    imp_prob2 = (1 / decimal_odds_2) * 100

    total_implied_prob = round(imp_prob1 + imp_prob2, 4)
    fair_prob1 = round(imp_prob1 / total_implied_prob * 100, 2)
    fair_prob2 = round(imp_prob2 / total_implied_prob * 100, 2)

    ## 47, 53 == 100
    ## 53 - 50 = 3

    return max(fair_prob1, fair_prob2) - 50

In [10]:
## each run takes 10 API calls currently (1 for getting games, 1 for underdog, 8 for sportsbooks)
bets = pd.DataFrame(columns = ['participant_name', 'ev', 'market', 'line'])

## markets we want to check
MARKETS = ['player_rebounds_over_under', 'player_points_rebounds_over_under', \
           'player_points_over_under', 'player_blocks_steals_over_under', \
            'player_blocks_over_under', 'player_assists_rebounds_over_under', \
            'player_assists_rebounds_over_under', 'player_assists_points_rebounds_over_under', \
            'player_assists_points_over_under', 'player_assists_over_under']

## steals, 

## get games
games = get_nba_games()
game_ids = [i['game_id'] for i in games['games']]

In [11]:
for market in MARKETS:
    for game in game_ids:
        try:
            odds = get_fantasy_lines(game, market)
            print(odds)
            ud = pd.DataFrame.from_dict(odds['fantasy_books'][2]['market']['lines'])
            ud = ud[['participant_name', 'line']]
            ud.columns = [ud.columns[0], 'handicap']

            ## now get all the sportsbooks lines
            odds = get_most_recent_odds(game, market)


            zero = pd.DataFrame.from_dict(odds['sportsbooks'][0]['market']['outcomes'])
            one = pd.DataFrame.from_dict(odds['sportsbooks'][1]['market']['outcomes'])
            two = pd.DataFrame.from_dict(odds['sportsbooks'][2]['market']['outcomes'])
            three = pd.DataFrame.from_dict(odds['sportsbooks'][3]['market']['outcomes'])
            five = pd.DataFrame.from_dict(odds['sportsbooks'][5]['market']['outcomes'])
            six = pd.DataFrame.from_dict(odds['sportsbooks'][6]['market']['outcomes'])
            seven = pd.DataFrame.from_dict(odds['sportsbooks'][7]['market']['outcomes'])
            eight = pd.DataFrame.from_dict(odds['sportsbooks'][8]['market']['outcomes'])

            ## process them
            zero = process_zero(zero)
            one = process_one(one)
            two = process_two_six(two)
            three = process_three_five(three)
            five = process_three_five(five)
            six = process_two_six(six)
            seven = process_seven_eight(seven)
            eight = process_seven_eight(eight)

            ## concat all the books
            datasets = [zero, one, two, three, five, six, seven, eight]
            new_datasets = []

            for dataset in datasets:
                dataset = pd.merge(dataset, ud, how='inner', on = ['participant_name', 'handicap'])
                new_datasets.append(pivot(dataset))

            concat = pd.concat(new_datasets, axis=0)
            final = concat.copy()

            final = final[final['ev'] >= 4.99]

            final['market'] = market

            if final.shape[0] > 0: bets = pd.concat([bets, final[['participant_name', 'ev', 'market', 'line']]])
            
        except Exception as e:
            print("exception", e, market)
            continue

{'game_id': 'c7e8a6cd9170e9d85be95112a749fb2a', 'fantasy_books': [{'bookie_key': 'draftkings', 'market': {'market_key': 'player_rebounds_over_under', 'lines': [{'timestamp': '2024-01-27T13:55:02.008159', 'game_id': 'c7e8a6cd9170e9d85be95112a749fb2a', 'line': 2.5, 'participant': 413, 'participant_name': 'Tyus Jones'}, {'timestamp': '2024-01-27T13:55:02.008738', 'game_id': 'c7e8a6cd9170e9d85be95112a749fb2a', 'line': 7.5, 'participant': 16126, 'participant_name': 'Isaiah Stewart'}, {'timestamp': '2024-01-27T13:55:02.007740', 'game_id': 'c7e8a6cd9170e9d85be95112a749fb2a', 'line': 6.5, 'participant': 16168, 'participant_name': 'Kyle Kuzma'}, {'timestamp': '2024-01-27T13:55:02.007396', 'game_id': 'c7e8a6cd9170e9d85be95112a749fb2a', 'line': 7.5, 'participant': 16730, 'participant_name': 'Daniel Gafford'}, {'timestamp': '2024-01-27T13:55:02.009060', 'game_id': 'c7e8a6cd9170e9d85be95112a749fb2a', 'line': 12.5, 'participant': 16762, 'participant_name': 'Jalen Duren'}, {'timestamp': '2024-01-27T1

In [12]:
bets.shape

(0, 4)

In [78]:
bets.to_csv("bets.csv", index=None)